In [34]:
from datetime import datetime, timedelta

# === INPUTS ===

user_preferences = {
    "optimalFocusDuration": "30–45 minutes",  # User's preferred focus duration
    "breakDuration": 5  # Break duration in minutes after each session (adjustable)
}

availability = {
    "12/02/2025": ["09:00-11:00", "14:00-16:00"],
    "14/02/2025": ["10:00-12:00"],
    "15/02/2025": ["09:00-11:00", "14:00-16:00"]
}

courses = [
    {
        "name": "Information Retrieval",
        "topics": [
            {"name": "Vector Space Model", "difficulty": 3, "understanding": 5, "studyTime": 60},  # studyTime in minutes
            {"name": "BM25", "difficulty": 4, "understanding": 2, "studyTime": 90}
        ]
    },
    {
        "name": "Software Architecture",
        "topics": [
            {"name": "Introduction", "difficulty": 1, "understanding": 4, "studyTime": 45},
            {"name": "System Architecture", "difficulty": 4, "understanding": 1, "studyTime": 120}
        ]
    }
]

# === HELPERS ===

def get_focus_minutes(pref):
    """Return focus duration in minutes based on preference"""
    if pref == "~30 minutes":
        return 30
    elif pref == "30–45 minutes":
        return 45
    elif pref == "1–2 hours":
        return 90
    elif pref == "Over 2 hours":
        return 120
    else:
        return 30  # default fallback

def estimate_sessions(difficulty, understanding, study_time):
    """Estimate number of sessions based on difficulty, understanding, and study time"""
    return max(1, round(study_time / get_focus_minutes(user_preferences["optimalFocusDuration"])))

def generate_time_slots(availability, focus_minutes, break_minutes):
    """Generate all possible time slots within the available time blocks, considering breaks"""
    slots = []
    for day, blocks in availability.items():
        for block in blocks:
            start, end = block.split("-")
            t_start = datetime.strptime(f"{day} {start}", "%d/%m/%Y %H:%M")
            t_end = datetime.strptime(f"{day} {end}", "%d/%m/%Y %H:%M")
            while t_start + timedelta(minutes=focus_minutes) <= t_end:
                slot_start = t_start.strftime("%d/%m/%Y %H:%M")
                slot_end = (t_start + timedelta(minutes=focus_minutes)).strftime("%d/%m/%Y %H:%M")
                slots.append((slot_start, slot_end))
                # Add break time after each session
                t_start += timedelta(minutes=focus_minutes + break_minutes)
    return slots

# === CSP VARIABLES ===

variables = []
domain = generate_time_slots(availability, get_focus_minutes(user_preferences["optimalFocusDuration"]), user_preferences["breakDuration"])
focus_minutes = get_focus_minutes(user_preferences["optimalFocusDuration"])

# Define sessions
for course in courses:
    for topic in course["topics"]:
        # Use the topic's own study time (studyTime) rather than estimating it from difficulty
        num = estimate_sessions(topic["difficulty"], topic["understanding"], topic["studyTime"])
        for i in range(1, num + 1):
            variables.append(f"{course['name']} - {topic['name']} - S{i}")

# === FUNCTIONALITY ===

# Tracking available time slots
occupied_slots = set()

def assign_time_slot(session_name):
    """Assign the next available time slot to a session and return the start and end time"""
    for slot_start, slot_end in domain:
        if slot_start not in occupied_slots:
            occupied_slots.add(slot_start)  # Mark this slot as taken
            return slot_start, slot_end
    return None  # If no slot available

# === ASSIGN TIME SLOTS ===

study_plan = {}

for session in variables:
    slot = assign_time_slot(session)
    if slot:
        study_plan[session] = slot
    else:
        print(f"❌ No valid time slot available for {session}")

# === OUTPUT ===

if study_plan:
    print("Study Plan:")
    for key in sorted(study_plan):
        start_time, end_time = study_plan[key]
        print(f"{key} → Start: {start_time}, End: {end_time}")
else:
    print("No valid study plan found.")

Study Plan:
Information Retrieval - BM25 - S1 → Start: 12/02/2025 09:50, End: 12/02/2025 10:35
Information Retrieval - BM25 - S2 → Start: 12/02/2025 14:00, End: 12/02/2025 14:45
Information Retrieval - Vector Space Model - S1 → Start: 12/02/2025 09:00, End: 12/02/2025 09:45
Software Architecture - Introduction - S1 → Start: 12/02/2025 14:50, End: 12/02/2025 15:35
Software Architecture - System Architecture - S1 → Start: 14/02/2025 10:00, End: 14/02/2025 10:45
Software Architecture - System Architecture - S2 → Start: 14/02/2025 10:50, End: 14/02/2025 11:35
Software Architecture - System Architecture - S3 → Start: 15/02/2025 09:00, End: 15/02/2025 09:45
